In [3]:
import tensorflow as tf

import numpy as np
import os
import datetime

In [4]:
import os
path = os.getcwd()
print(path)

/content


In [8]:

from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  


Saving Tolstoy_Lev.txt to Tolstoy_Lev.txt
User uploaded file "Tolstoy_Lev.txt" with length 5353805 bytes


In [9]:
import glob
import codecs
books = sorted(glob.glob(path + "/*.txt"))
print("Found {} books".format(len(books)))

text = ""
for filename in books:
    with codecs.open(filename, 'r', 'utf-8') as books:
        text += books.read()

print("Text is {} characters long".format(len(text)))

Found 2 books
Text is 3121282 characters long


In [15]:
words = [w for w in text.split(' ') if w.strip() != '' or w == '\n']
print("Text is {} words long".format(len(words)))

Text is 466630 words long


In [16]:
print(text[:100])

Я покидаю дом в семь часов утра
В одном кармане диплом , а в другом дыра
Я сижу курю на детской пл


In [17]:
vocab = sorted(set(text))
print ('There are {} unique characters'.format(len(vocab)))
char2int = {c:i for i, c in enumerate(vocab)}
int2char = np.array(vocab)
print('Vector:\n')
for char,_ in zip(char2int, range(len(vocab))):
   print(' {:4s}: {:3d},'.format(repr(char), char2int[char]))

There are 180 unique characters
Vector:

 '\n':   0,
 '\r':   1,
 ' ' :   2,
 '!' :   3,
 '"' :   4,
 '$' :   5,
 "'" :   6,
 '(' :   7,
 ')' :   8,
 '*' :   9,
 ',' :  10,
 '-' :  11,
 '.' :  12,
 '/' :  13,
 '0' :  14,
 '1' :  15,
 '2' :  16,
 '3' :  17,
 '4' :  18,
 '5' :  19,
 '6' :  20,
 '7' :  21,
 '8' :  22,
 '9' :  23,
 ':' :  24,
 ';' :  25,
 '<' :  26,
 '=' :  27,
 '>' :  28,
 '?' :  29,
 'A' :  30,
 'B' :  31,
 'C' :  32,
 'D' :  33,
 'E' :  34,
 'F' :  35,
 'G' :  36,
 'H' :  37,
 'I' :  38,
 'J' :  39,
 'K' :  40,
 'L' :  41,
 'M' :  42,
 'N' :  43,
 'O' :  44,
 'P' :  45,
 'Q' :  46,
 'R' :  47,
 'S' :  48,
 'T' :  49,
 'U' :  50,
 'V' :  51,
 'W' :  52,
 'X' :  53,
 'Y' :  54,
 'Z' :  55,
 '[' :  56,
 ']' :  57,
 'a' :  58,
 'b' :  59,
 'c' :  60,
 'd' :  61,
 'e' :  62,
 'f' :  63,
 'g' :  64,
 'h' :  65,
 'i' :  66,
 'j' :  67,
 'k' :  68,
 'l' :  69,
 'm' :  70,
 'n' :  71,
 'o' :  72,
 'p' :  73,
 'q' :  74,
 'r' :  75,
 's' :  76,
 't' :  77,
 'u' :  78,
 'v' :  79,

In [13]:
text_as_int = np.array([char2int[ch] for ch in text], dtype=np.int32)
print ('{}\n mapped to integers:\n {}'.format(repr(text[:100]), text_as_int[:100]))

'Я покидаю дом в семь часов утра\r\nВ одном кармане диплом , а в другом дыра\r\nЯ сижу курю на детской пл'
 mapped to integers:
 [138   2 154 153 149 147 143 139 169   2 143 153 151   2 141   2 156 144
 151 167   2 162 139 156 153 141   2 158 157 155 139   1   0 112   2 153
 143 152 153 151   2 149 139 155 151 139 152 144   2 143 147 154 150 153
 151   2  10   2 139   2 141   2 143 155 158 142 153 151   2 143 166 155
 139   1   0 138   2 156 147 145 158   2 149 158 155 169   2 152 139   2
 143 144 157 156 149 153 148   2 154 150]


In [18]:
tr_text = text_as_int[:2500000] 
val_text = text_as_int[2500000:] 

In [19]:
print(text_as_int.shape, tr_text.shape, val_text.shape)

(3121282,) (2500000,) (621282,)


In [20]:
batch_size = 64
buffer_size = 10000
embedding_dim = 256
epochs = 15
seq_length = 200
examples_per_epoch = len(text)//seq_length

rnn_units = 1024
vocab_size = len(vocab)

In [21]:
tr_char_dataset = tf.data.Dataset.from_tensor_slices(tr_text)
val_char_dataset = tf.data.Dataset.from_tensor_slices(val_text)
print(tr_char_dataset, val_char_dataset)
tr_sequences = tr_char_dataset.batch(seq_length+1, drop_remainder=True)
val_sequences = val_char_dataset.batch(seq_length+1, drop_remainder=True)
print(tr_sequences, val_sequences)

for item in tr_sequences.take(1):
    print(repr(''.join(int2char[item.numpy()])))
    print(item)
for item in val_sequences.take(1):
    print(repr(''.join(int2char[item.numpy()])))
    print(item)

<TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.int32, name=None)> <TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.int32, name=None)>
<BatchDataset element_spec=TensorSpec(shape=(201,), dtype=tf.int32, name=None)> <BatchDataset element_spec=TensorSpec(shape=(201,), dtype=tf.int32, name=None)>
'Я покидаю дом в семь часов утра\r\nВ одном кармане диплом , а в другом дыра\r\nЯ сижу курю на детской площадке\r\nЯ в детстве о счастье мечтал, но надежды так шатки\r\nСиноптик-паразит: вместо солнца дождик мо'
tf.Tensor(
[138   2 154 153 149 147 143 139 169   2 143 153 151   2 141   2 156 144
 151 167   2 162 139 156 153 141   2 158 157 155 139   1   0 112   2 153
 143 152 153 151   2 149 139 155 151 139 152 144   2 143 147 154 150 153
 151   2  10   2 139   2 141   2 143 155 158 142 153 151   2 143 166 155
 139   1   0 138   2 156 147 145 158   2 149 158 155 169   2 152 139   2
 143 144 157 156 149 153 148   2 154 150 153 164 139 143 149 144   1   0
 138   2 141   

In [22]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

tr_dataset = tr_sequences.map(split_input_target).shuffle(buffer_size).batch(batch_size, drop_remainder=True)
val_dataset = val_sequences.map(split_input_target).shuffle(buffer_size).batch(batch_size, drop_remainder=True)
print(tr_dataset, val_dataset)

<BatchDataset element_spec=(TensorSpec(shape=(64, 200), dtype=tf.int32, name=None), TensorSpec(shape=(64, 200), dtype=tf.int32, name=None))> <BatchDataset element_spec=(TensorSpec(shape=(64, 200), dtype=tf.int32, name=None), TensorSpec(shape=(64, 200), dtype=tf.int32, name=None))>


In [23]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.LSTM(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dropout(0.2), 
        tf.keras.layers.LSTM(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [24]:
model = build_model(
    vocab_size = len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=batch_size)

In [25]:
for input_example_batch, target_example_batch in tr_dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "respectively: batch_size, sequence_length, vocab_size")

(64, 200, 180) respectively: batch_size, sequence_length, vocab_size


In [26]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (64, None, 256)           46080     
                                                                 
 dropout (Dropout)           (64, None, 256)           0         
                                                                 
 lstm (LSTM)                 (64, None, 1024)          5246976   
                                                                 
 dropout_1 (Dropout)         (64, None, 1024)          0         
                                                                 
 lstm_1 (LSTM)               (64, None, 1024)          8392704   
                                                                 
 dropout_2 (Dropout)         (64, None, 1024)          0         
                                                                 
 dense (Dense)               (64, None, 180)           1

In [27]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)
def accuracy(labels, logits):
    return tf.keras.metrics.sparse_categorical_accuracy(labels, logits)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
example_batch_acc  = accuracy(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Loss:      ", example_batch_loss.numpy().mean())
print("Accuracy:      ", example_batch_acc.numpy().mean())

Prediction shape:  (64, 200, 180)  # (batch_size, sequence_length, vocab_size)
Loss:       5.1927977
Accuracy:       0.007734375


In [28]:
optimizer = tf.keras.optimizers.Adam() 
model.compile(optimizer=optimizer, loss=loss) 

In [29]:
checkpoint_dir = './checkpoints'+ datetime.datetime.now().strftime("_%Y.%m.%d-%H:%M:%S")
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [31]:
patience = 10
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=patience)

In [32]:
history = model.fit(tr_dataset, epochs=epochs, callbacks=[checkpoint_callback, early_stop] , validation_data=val_dataset)
print ("Training stopped as there was no improvement after {} epochs".format(patience))

Epoch 1/15
194/194 [==============================] - 74s 352ms/step - loss: 2.9630 - val_loss: 2.3029
Epoch 2/15
194/194 [==============================] - 74s 372ms/step - loss: 2.0623 - val_loss: 1.7496
Epoch 3/15
194/194 [==============================] - 70s 351ms/step - loss: 1.7087 - val_loss: 1.5466
Epoch 4/15
194/194 [==============================] - 70s 350ms/step - loss: 1.5526 - val_loss: 1.4546
Epoch 5/15
194/194 [==============================] - 70s 350ms/step - loss: 1.4616 - val_loss: 1.3970
Epoch 6/15
194/194 [==============================] - 71s 353ms/step - loss: 1.3998 - val_loss: 1.3668
Epoch 7/15
194/194 [==============================] - 70s 352ms/step - loss: 1.3518 - val_loss: 1.3443
Epoch 8/15
194/194 [==============================] - 71s 353ms/step - loss: 1.3118 - val_loss: 1.3275
Epoch 9/15
194/194 [==============================] - 71s 353ms/step - loss: 1.2769 - val_loss: 1.3149
Epoch 10/15
194/194 [==============================] - 70s 350ms/step - l

KeyboardInterrupt: ignored

In [34]:
tf.train.latest_checkpoint(checkpoint_dir)

'./checkpoints_2022.09.10-13:06:51/ckpt_12'

In [35]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))  
model.build(tf.TensorShape([1, None]))
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (1, None, 256)            46080     
                                                                 
 dropout_3 (Dropout)         (1, None, 256)            0         
                                                                 
 lstm_2 (LSTM)               (1, None, 1024)           5246976   
                                                                 
 dropout_4 (Dropout)         (1, None, 1024)           0         
                                                                 
 lstm_3 (LSTM)               (1, None, 1024)           8392704   
                                                                 
 dropout_5 (Dropout)         (1, None, 1024)           0         
                                                                 
 dense_1 (Dense)             (1, None, 180)           

In [36]:
def generate_text(model, start_string):
    
    print('Generating with seed: "' + start_string + '"')
  
    num_generate = 1000

    input_eval = [char2int[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    text_generated = []

    temperature = 1.0

    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
        input_eval = tf.expand_dims([predicted_id], 0)
        text_generated.append(int2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [37]:
print(generate_text(model, start_string="Был в кино с Наташей Ростовой"))

Generating with seed: "Был в кино с Наташей Ростовой"
Был в кино с Наташей Ростовой подманил с каждым домом, и главноуl– Ей-до намиились и грозный вопрос, качило и тихо напрягал и опять прижимался за жилето по лестнице, уже встал и видимо подвинулся набок спинку набок к радостной.
Князь Андрей, одевшись по листу с выражением лица, голову, который выскажет на пятисутилеескую) принцу с Перодионом и, уже объявия его, крестящий смех, пят, под остроумным небо, как опрядел к своему товарищу, – сказал Дихани по гостиной.
– Вы в доме (тогда же, как она просит машу спинку отца.
После обеда под совершившемуся, произнесшей узабрежением выражение блызна в сердце Наташи, почтительно просилу несколько позволения. Он с очень тертвым, чувствовал, что как будто он так неделен чувствовал, что формально говорил Жерков, и два человека завещания и часто торопящие и воображает этот мест и старый князь проходил по воропаха времени совершенствовавшего было в душе своей с другими лица.
Когда стояли у ней потом